In [1]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
from nselib import capital_market
import yfinance as yf
import math
import os
import numpy as np
import pandas as pd
from datetime import datetime, date
import warnings
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
# from flask_ngrok import run_with_ngrok
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pickle
import sys

warnings.filterwarnings("ignore")
app = Flask(__name__)
CORS(app) 

# run_with_ngrok(app)

global sym, stock, rec, ebitda_margin, teps, feps, f_pe, t_pe
@app.route('/', methods=['POST'])
def get_stock_information():
    if request.method == 'POST':
        global sym
        sym = request.form['symbol']
        sym2 = str(sym) + '.NS'
        global stock
        stock = yf.Ticker(sym2)
        info_dict = stock.info
        key_list = ['symbol', 'quoteType', 'longName', 'city', 'website', 'industry', 'fullTimeEmployees',
                    'previousClose', 'open', 'dayLow', 'dayHigh', 'fiftyTwoWeekHigh', 'dividendRate',
                    'dividendYield', 'marketCap', 'trailingPE', 'forwardPE', 'trailingEps', 'forwardEps',
                    'enterpriseToEbitda', 'debtToEquity', 'returnOnEquity', 'revenuePerShare', 'ebitdaMargins',
                    'currentPrice', 'targetHighPrice', 'targetLowPrice', 'recommendationKey']
        new_info_dict = {}
        counter = 0
        for key, value in info_dict.items():
            if key in key_list:
                counter += 1
                new_info_dict[key] = value
        df = pd.DataFrame([new_info_dict])
        global rec
        rec = new_info_dict.get('recommendationKey', '')
        quoteType = new_info_dict.get('quoteType', '')
        longName = new_info_dict.get('longName', '')
        fiftyTwoWeekHigh = new_info_dict.get('fiftyTwoWeekHigh',0.0)
        dividendRate = new_info_dict.get('dividendRate',0.0)
        dividendYield = new_info_dict.get('dividendYield',0.0)
        marketCap = new_info_dict.get('marketCap',0.0)
        debtToEquity = new_info_dict.get('debtToEquity',0.0)
        returnOnEquity = new_info_dict.get('returnOnEquity',0.0)
        global ebitda_margin,teps,feps,f_pe,t_pe
        ebitda_margin = new_info_dict.get('ebitdaMargins', 0.0)
        teps = new_info_dict.get('trailingEps', 0.0)
        feps = new_info_dict.get('forwardEps', teps)
        t_pe = new_info_dict.get('trailingPE', 0.0)
        f_pe = new_info_dict.get('forwardPE', t_pe)
        industry = new_info_dict.get('industry', '')

        # Prepare the response data in JSON format
        response_data = {
            'symbol': sym,
            'quoteType': quoteType,
            'longName' : longName,
            'industry' : industry,
            'recommendation': rec,
            'fiftyTwoWeekHigh' : fiftyTwoWeekHigh,
            'dividendRate' : dividendRate,
            'dividendYield' : dividendYield,
            'marketCap' : marketCap,
            'debtToEquity' : debtToEquity,
            'returnOnEquity' : returnOnEquity,
            'ebitdaMargins': ebitda_margin,
            'trailingEps': teps,
            'forwardEps': feps,
            'trailingPE': t_pe,
            'forwardPE': f_pe
        }

        return jsonify(response_data) 

@app.route('/forecast', methods=['POST'])
def forecast():
    global sym
    print(sym)
    fut_pred,new_df = button_clicked()
    
    response_dict = {
        'predictions': fut_pred.tolist(),              # Convert pandas Series to list
        'dataframe': new_df.to_dict(orient='records')  # Converts pandas Dataframe to list of dict
    }
    return jsonify(response_dict)

global train_set, test_set, future
def button_clicked():
    data = capital_market.price_volume_data(symbol=sym, period='1Y')
    df = pd.DataFrame({'Date':data['Date'], 'Close':data['ClosePrice']})
    l = len(df)
    flag = 1
    for i in range(l):
       if type(df['Close'][i]) != np.float64:
        flag = 0
    if flag == 0:
        df['Close'] = df['Close'].str.replace(',', '').astype(float)
    if flag == 1:
        df['Close'] = df['Close'].replace(',', '').astype(float)
    stepwise_fit = auto_arima(df['Close'], suppress_warnings=True)
    stepwise_fit.summary()

    n = len(df['Close'])
    train_size = int(n * 0.80)
    #train test split
    global train_set
    train_set = df.iloc[:train_size]
    global test_set
    test_set = df.iloc[train_size:]
    print(len(train_set))
    print(len(test_set))

    model = ARIMA(df['Close'], order = (31,2,31))
    model_fit = model.fit()
    start_point = len(train_set)
    end_point = len(train_set) + len(test_set) - 1
    pred = model_fit.predict(start= start_point, end = end_point, type='levels').rename('ARIMA predictions')
# pred.plot(legend=True)
# test_set['Close'].plot(legend=True)

    mse = mean_squared_error(pred, test_set['Close'])
    print("MSE = ",mse)
    rmse = math.sqrt(mse)
    print("RMSE = ",rmse)
    #Future trend 
    global future
    future = model_fit.predict(start= len(df), end= len(df)+50, type='levels').rename("Forecast")
# print(future)

    plt.figure(figsize=(8,6))
    df['Close'].plot(color = 'blue', legend=True)
# future.plot(legend=True, title=f"{sym} 50-Day Forecast")
    flg = 0
    if future[len(df) + 50] > future[len(df)]:
        flg = 1
    if flg == 1:
        future.plot(color='#5dbd2d', legend=True, title=f"{sym} 50-Day Forecast")
    else:
        future.plot(color = 'red', legend=True, title=f"{sym} 50-Day Forecast")
        
    
    # Save the plot as an image
    image_path = os.path.join(app.static_folder, 'forecast_plot.png')
    plt.savefig(image_path)
    plt.close()

    #sys.stdout.flush()
    #print(df)

    week_df = df.loc[df.index % 7 == 0,['Date','Close']]
    week_df.reset_index(drop=True, inplace=True)
    #print(week_df)

    return future,week_df


@app.route("/buysell",methods=['POST'])
def buysell():
    weight_sum = 0
    global stock
    bal_sheet = stock.balance_sheet
    row_index='Total Capitalization'
    w=0
    #weight = 3
    c = len(bal_sheet.columns)
    print(c)
    for i in range(c):
        if (float(bal_sheet.loc[row_index,bal_sheet.columns[i]]) - float(bal_sheet.loc[row_index,bal_sheet.columns[i]])) > 0:
            w += 1
        else:
            w -= 1
    print(w)
    weight_sum += w
    
    #weight = 3
    row_index='Total Assets'
    w=0
    c = len(bal_sheet.columns)
    print(c)
    for i in range(c):
        if (float(bal_sheet.loc[row_index,bal_sheet.columns[i]]) - float(bal_sheet.loc[row_index,bal_sheet.columns[i]])) > 0:
            w += 1
        else:
            w -= 1
    print(w)
    weight_sum += w
    global train_set,test_set
    s0 = len(train_set) + len(test_set)
    s1 = s0 + 10
    s2 = s0 + 20
    s3 = s0 + 30
    s4 = s0 + 40
    s5 = s0 + 50
    def slope(x,y):
        global future
        m = (future[y] - future[x])/ (y-x)
        return m
    m1 = slope(s0,s1)
    m2 = slope(s0,s2)
    m3 = slope(s0,s3)
    m4 = slope(s0,s4)
    m5 = slope(s0,s5)
    fc = (m1 + m2 + m3 + m4 + m5)/5

#weight = 30
    if (fc > 0) and (fc <= 1):
        weight_sum += 30
    elif (fc > 1):
        weight_sum += 60
    if (fc < 0) and (fc >= -0.5):
        weight_sum -= 30
    elif (fc < -0.5):
        weight_sum -= 60  
        
    #weight = 5
    global rec
    if(rec == 'strong_buy'):
        weight_sum+=10
    elif (rec == 'buy'):
        weight_sum+=5
    else:
        weight_sum-=10

    #weight = 5
    global f_pe,t_pe
    if (f_pe > t_pe):
        weight_sum += 5
    else:
        weight_sum -= 5
        
    #weight = 3
    pe = (f_pe + t_pe)/2
    if (pe > 10)and(pe < 25):
        weight_sum += 3
    else:
        weight_sum -= 3
        
    #weight = 5
    global feps,teps
    if feps > teps:
        weight_sum += 5
    else:
        weight_sum -= 5
        
    #weigth = 3
    global ebitda_margin
    if ebitda_margin > 0.10 :
        weight_sum += 3
    else:
        weight_sum -= 3
    
    #weight = 3
    cash_flo=stock.cash_flow
    w=0
    c = len(cash_flo.columns)
    print(c)
    row_index='Free Cash Flow'
    for i in range(c):
        v = float(cash_flo.loc[row_index, cash_flo.columns[i]])
        if v > 0:
            w += 3
        else:
            w-=3
    weight_sum += w

    F_ind = (weight_sum)/61
    recommendation = ""

    if F_ind <= -0.15:
        recommendation = "SELL"
    elif (F_ind > -0.15) and (F_ind < 0.15):
        recommendation = "HOLD"
    elif (F_ind >= 0.15) and (F_ind < 0.50):
        recommendation = "BUY"
    elif(F_ind >= 0.50):
        recommendation = "STRONG BUY"
    print(F_ind)
    print(recommendation)
    print(stock)

    response_dict = {
        'rec_val' : F_ind,
        'buy_sell_ind' : recommendation
    }
    return jsonify(response_dict)

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Feb/2024 22:50:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:50:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:50:31] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 22:59:47] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:03:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:06:00] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:07:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:11:14] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:31:24] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:32:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:37:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:40:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:40:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:47:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 23:50:31] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2024 2

tcs
199
50
MSE =  1723.8093867636676
RMSE =  41.518783541472736


127.0.0.1 - - [08/Feb/2024 02:14:58] "POST /forecast HTTP/1.1" 200 -


tcs
199
50
MSE =  1723.8093867636676
RMSE =  41.518783541472736


...

127.0.0.1 - - [08/Feb/2024 02:26:51] "POST /forecast HTTP/1.1" 200 -


MSE =  1723.8093867636676
RMSE =  41.518783541472736
tcs
199
50
MSE =  1723.8093867636676
RMSE =  41.518783541472736


127.0.0.1 - - [08/Feb/2024 02:36:45] "POST /forecast HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2024 03:12:13] "POST / HTTP/1.1" 200 -


bpcl
198
50
MSE =  66.2185595391261
RMSE =  8.137478696692613


127.0.0.1 - - [08/Feb/2024 03:26:22] "POST /forecast HTTP/1.1" 200 -
